In [ ]:
# importar las librerías requeridas
import pandas as pd
import requests
import re
from io import StringIO


In [ ]:
#Importar librería de google drive para la carga de los archivos de datos requeridos.
#from google.colab import drive
#drive.mount('/content/drive')
#si se va a cargar desde google drive usar este comando
#path = "/content/drive/MyDrive/Trabajo/Datos/climatologicos/"


#si se va a cargar desde github se deben usar estos comandos
#Se clona el repositorio de Github donde se encuentran los datasets fuente
! git clone https://github.com/jestebanrojas/seminario.git
# Se crea un path específico para facilitar la lectura de los archivos posteriormente desde github
path = "/content/seminario/Datos/climatologicos/"


Mounted at /content/drive
Cloning into 'seminario'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 221 (delta 121), reused 212 (delta 112), pack-reused 0
Receiving objects: 100% (221/221), 15.15 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [ ]:
"""
Para la descarfa de la información climatológica histórica, se empleará la API de la NASA dispuesta para tal fin.
En este caso se descargará la información para todos los municipios de Antioquia.
Se toma como referencia las coordenadas del casco urbano de cada municipio de Antioquia, los cuales se han obtenido previamente del DANE
"""

#Leer archivo con las coordenadas de cada municipio, las cuales son necesarias para consultar en la nasa los valores de las variables climatológicas
coordenadas_df=pd.read_csv(path+'coordenadas.csv', sep=';')




In [ ]:
coordenadas_df


,Código Municipio,Código Centro Poblado,Nombre Departamento,Nombre Municipio,Nombre Centro Poblado,Tipo Centro Poblado,Longitud,Unnamed: 7,Latitud,Distrito,Tipo de Municipio,Area Metropolitana
0,5001,5001000,ANTIOQUIA,MEDELLIN,MEDELLIN,CABECERA MUNICIPAL (CM),-75.576002,NaN,6.248586,NaN,MUNICIPIO,AREA METROPOLITANA DEL VALLE DE ABURRÁ
1,5002,5002000,ANTIOQUIA,ABEJORRAL,ABEJORRAL,CABECERA MUNICIPAL (CM),-75.428740,NaN,5.789301,NaN,MUNICIPIO,NaN
2,5004,5004000,ANTIOQUIA,ABRIAQUI,ABRIAQUI,CABECERA MUNICIPAL (CM),-76.064295,NaN,6.632282,NaN,MUNICIPIO,NaN
3,5021,5021000,ANTIOQUIA,ALEJANDRIA,ALEJANDRIA,CABECERA MUNICIPAL (CM),-75.141334,NaN,6.376063,NaN,MUNICIPIO,NaN
4,5030,5030000,ANTIOQUIA,AMAGA,AMAGA,CABECERA MUNICIPAL (CM),-75.701948,NaN,6.038808,NaN,MUNICIPIO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120,5885,5885000,ANTIOQUIA,YALI,YALI,CABECERA MUNICIPAL (CM),-74.840053,NaN,6.676572,NaN,MUNICIPIO,NaN
121,5887,5887000,ANTIOQUIA,YARUMAL,YARUMAL,CABECERA MUNICIPAL (CM),-75.418833,NaN,6.963871,NaN,MUNICIPIO,NaN
122,5890,5890000,ANTIOQUIA,YOLOMBO,YOLOMBO,CABECERA MUNICIPAL (CM),-75.013348,NaN,6.594596,NaN,MUNICIPIO,NaN
123,5893,5893000,ANTIOQUIA,YONDO,YONDO,CABECERA MUNICIPAL (CM),-73.912457,NaN,7.003941,NaN,MUNICIPIO,NaN


In [ ]:
#Se empleará la API de la nada para la descarga de la información climatológica deseada
#Más información en https://power.larc.nasa.gov/data-access-viewer/ y https://power.larc.nasa.gov/docs/

# api-endpoint para la descarga de la información
URL = "https://power.larc.nasa.gov/api/temporal/hourly/point"

In [ ]:
#Crear un dataframe vacío, el cual se usará para ir anexando la consulta de cada municipio en un solo dataframe
df_total = pd.DataFrame()

#Se iterará por cada municipio, para realizar la descarga de datos por municipio
for municipio in coordenadas_df.index:
  #Cargar las coordenadas del municipio en las variables longitude y latitude
  longitude=coordenadas_df['Longitud'][municipio]
  latitude=coordenadas_df['Latitud'][municipio]

  #Guardar el nombre del municiío para anexarlo en una columna del dataframe posteriormente
  municipio=coordenadas_df['Nombre Municipio'][municipio]

  # Definición de los parámetros para la descarga de los datos (Variables a descargar, rango de fechas, coordenadas y formato)
  PARAMS = {'Time': 'LST', 'parameters': 'ALLSKY_SFC_SW_DWN,ALLSKY_KT,T2M,RH2M,PRECTOTCORR,CLRSKY_SFC_SW_DWN,T2MWET','community':'RE','longitude':longitude,'latitude':latitude,'start':'20180101','end':'20231031','format':'csv'}
  print("\n -------------------- \n")
  print("\n -------------------- \n")
  print("Descargando datos meteorológicos del municipio: ")
  print("Municipio",municipio, "Latitud: ", latitude, "Longitud: ",longitude)
  print("\n -------------------- \n")
  print("\n -------------------- \n")

  # realizar el GET request y almacenarlo en r
  r=requests.get(url = URL, params = PARAMS)

  #Debido al formato de descarga, el cual contiene un encabezado y los datos propiamente, se requiere separar el encabezaod de los datos
  #Dicha separación de realiza a través del método split, teniendo como string separador "-END HEADER-\r\n"
  x = re.split("-END HEADER-\r\n", r.text, 1)
  #Solo se toman los datos, los cuales se almacenan en la posición 1. Se emplea la función StringIO para poder ser leida posteriormente
  #como formato cssv y convertirlo a dataframe
  environment_data = StringIO(x[1])
  #Conversión de los datos climatológicos del municipio en un dataframe
  env_df=pd.read_csv(environment_data)
  #Se anexa al dataframe una columna con el nombre del municipio
  env_df['Municipio']=municipio
  #se anexa el dataframe con la información específica del municipio al datafreme que acumula toda la información de todos los municipios.
  df_total=pd.concat([df_total,env_df],sort=False)

print(df_total)


Municipio MEDELLIN Latitud:  6.248586367 Longitud:  -75.57600171
Municipio ABEJORRAL Latitud:  5.789301114 Longitud:  -75.42874039
Municipio ABRIAQUI Latitud:  6.632281955 Longitud:  -76.06429487
Municipio ALEJANDRIA Latitud:  6.376062643 Longitud:  -75.1413345
Municipio AMAGA Latitud:  6.038808424 Longitud:  -75.70194834
Municipio AMALFI Latitud:  6.909673605 Longitud:  -75.07747492
Municipio ANDES Latitud:  5.656769713 Longitud:  -75.87879408
Municipio ANGELOPOLIS Latitud:  6.109612778 Longitud:  -75.71135338
Municipio ANGOSTURA Latitud:  6.885184415 Longitud:  -75.33510912
Municipio ANORI Latitud:  7.074710041 Longitud:  -75.14834496
Municipio SANTA FE DE ANTIOQUIA Latitud:  6.55653286 Longitud:  -75.82707832
Municipio ANZA Latitud:  6.302666764 Longitud:  -75.85444876
Municipio APARTADO Latitud:  7.883251082 Longitud:  -76.62589092
Municipio ARBOLETES Latitud:  8.849918273 Longitud:  -76.42665816
Municipio ARGELIA Latitud:  5.731479228 Longitud:  -75.14106336
Municipio ARMENIA Lati

In [ ]:
#Verificar si existe directorio en el entorno
import os
path = "Datasets_salida/climatologicos/"
# verificación de si existe o no el directorio
isExist = os.path.exists(path)
if not isExist:

   # Ccrea el directorio cuando no existe
   os.makedirs(path)
   print("\n -------------------- \n")
   print("\n -------------------- \n")
   print("se creó el directorio: ", path)
   print("\n -------------------- \n")
   print("\n -------------------- \n")

#Se guarada el dataframe en un archivo csv para ser leido posteriormente por otros notebooks
#df_total.to_csv('/content/drive/MyDrive/Trabajo/Datasets_salida/climatologicos/clima.csv', index=False)


df_total.to_csv('/content/Datasets_salida/climatologicos/clima.csv', index=False)

print("\n -------------------- \n")
print("\n -------------------- \n")
print("se creó el archivo: clima.csv")
print("\n -------------------- \n")
print("\n -------------------- \n")